In [1]:
import geopandas as gpd
import requests
import pandas as pd

url = 'https://opendata.arcgis.com/datasets/490db54636704d35aae661a12c12e9a0_0.geojson'
r = requests.get(url, allow_redirects=True)
open('Bus_Stop_Inventory.geojson', 'wb').write(r.content)
geo_df = gpd.read_file('Bus_Stop_Inventory.geojson')

In [24]:
# geo_df.loc[geo_df['trapeze_id'].isnull(),]['trapeze_status'].value_counts()

In [29]:
import pandas as pd
import glob

allFiles = glob.glob('.' + "/results/*.csv")
for file_ in allFiles:
    df = pd.read_csv(file_)
    df = df.query("TIME_PERIOD=='AM Peak'|TIME_PERIOD=='PM Peak'")
    df = df.sort_values(by=['TIME_PERIOD','DIRECTION_NAME','SORT_ORDER'])
    df['delay_secs_mean'] = df['travel_time_secs_mean'] - df['travel_time_min_secs']
    df['activity'] = df['BOARD_ALL'] + df['ALIGHT_ALL']
    df['sec_per_activity'] = df['activity']/df['dwell_time_secs_mean']
    df['sec_per_delay'] = df['LOAD_ALL']/df['dwell_time_secs_mean'].shift(periods=-1)
    df = pd.merge(df,geo_df[['trapeze_id','stopname']],left_on=['STOP_ID'],right_on=['trapeze_id'])
    df = df.sort_values(by=['TIME_PERIOD','DIRECTION_NAME','SORT_ORDER'])
    columns_to_keep = ['STOP_ID','stopname','TIME_PERIOD','DIRECTION_NAME','SORT_ORDER','travel_time_secs_mean',
                       'travel_speed_miles_per_hour','travel_time_secs_std','travel_time_min_secs', 'delay_secs_mean','dwell_time_secs_mean',
                       'dwell_time_secs_std','BOARD_ALL','ALIGHT_ALL', 'LOAD_ALL', 'activity','sec_per_delay', 'AVG_SERVICED', 'sec_per_activity', 'percent_stopped']
    df[columns_to_keep].to_csv("fast_output/fast_output_" + file_.split('_')[-1], index=False)